In [41]:
%matplotlib notebook
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.colors as mcol
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import math

products = pd.read_csv('products.txt', header=None) #If header is not included the 1st line is automatically considered
# as column names
products.columns = ['product_name','manufacturer','model','family','announced date']

for index, row in products.iterrows():    
    products.loc[index, "product_name"] = products.loc[index, "product_name"].split('"')[3]
    products.loc[index, "manufacturer"] = products.loc[index, "manufacturer"].split('"')[1]
    products.loc[index, "model"] = products.loc[index, "model"].split('"')[1]
    if 'family' in products.loc[index, "family"]:
        products.loc[index, "family"] = products.loc[index, "family"].split('"')[1]
    if 'announced' in products.loc[index, "family"]:
        products.loc[index, "announced date"] = products.loc[index, "family"]
        products.loc[index, "family"] = ''
    products.loc[index, "announced date"] = products.loc[index, "announced date"].split('"')[1]

products.columns=['product_name','manufacturer','model','family','announced date']
products.reset_index()

products.head(5)

,product_name,manufacturer,model,family,announced date
0,Sony_Cyber-shot_DSC-W310,Sony,DSC-W310,Cyber-shot,2010-01-06T19:00:00.000-05:00
1,Samsung_TL240,Samsung,TL240,,2010-01-05T19:00:00.000-05:00
2,Nikon-s6100,Nikon,S6100,Coolpix,2011-02-08T19:00:00.000-05:00
3,Samsung_TL220,Samsung,TL220,,2009-08-12T20:00:00.000-04:00
4,Fujifilm-T205,Fujifilm,T205,FinePix,2011-02-15T19:00:00.000-05:00


In [42]:
listings = pd.read_csv('listings.txt', error_bad_lines=False, warn_bad_lines=False, header=None) 

listings.columns = ["title", "manufacturer", "currency", "price"]
pd.options.display.max_colwidth=200

for index, row in listings.iterrows():
    listings.loc[index,'manufacturer']=listings.loc[index,'manufacturer'].split('"')[1]
    listings.loc[index,'currency']=listings.loc[index,'currency'].split('"')[1]
    listings.loc[index,'price']=listings.loc[index,'price'].split('"')[1]
    listings.loc[index,'title']=listings.loc[index,'title'].split('"')[3]


listings.head()

,title,manufacturer,currency,price
0,LED Flash Macro Ring Light (48 X LED) with 6 Adapter Rings for For Canon/Sony/Nikon/Sigma Lenses,Neewer Electronics Accessories,CAD,35.99
1,Canon PowerShot SX130IS 12.1 MP Digital Camera with 12x Wide Angle Optical Image Stabilized Zoom with 3.0-Inch LCD,Canon Canada,CAD,199.96
2,Canon PowerShot SX130IS 12.1 MP Digital Camera with 12x Wide Angle Optical Image Stabilized Zoom with 3.0-Inch LCD,Canon Canada,CAD,209.00
3,Canon PowerShot D10 12.1 MP Waterproof Digital Camera with 3x Optical Image Stabilized Zoom and 2.5-inch LCD (Blue/Silver),Canon Canada,CAD,306.24
4,Canon PowerShot D10 12.1 MP Waterproof Digital Camera with 3x Optical Image Stabilized Zoom and 2.5-inch LCD (Blue/Silver),Canon Canada,CAD,420.33


In [43]:
def match_strs(row):
    match_result = (np.where(listings.title.str.contains(row['manufacturer']) & \
                             listings.title.str.contains(row['family']) & \
                             listings.title.str.contains(row['model'])))
    if not len(match_result[0]):
        return None
    return match_result[0][0]

products['merge_idx'] = products.apply(match_strs, axis='columns')

C:\Users\Cyrus\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


In [60]:
output=pd.DataFrame(columns={0,1,2,3,4})
output=(products.merge(listings, left_on='merge_idx', right_on='index', right_index=True, how='right')
  .drop(['merge_idx','manufacturer_x','model','family','announced date'] , 1).dropna())
output.columns=["product_name", 'title', 'manufacturer', 'currency', 'price']
output=output.reset_index(drop='index')
output

,product_name,title,manufacturer,currency,price
0,Leica_Digilux,Leica 'Digilux 2' 5MP Digital Camera with 3.2x Optical Zoom,Leica,USD,1299.00
1,Leica_Digilux_Zoom,Leica 'Digilux 2' 5MP Digital Camera with 3.2x Optical Zoom,Leica,USD,1299.00
2,Fujifilm_FinePix_1500,Fujifilm FinePix S1500FD 10 MP 12x Zoom Digital Camera (Black),Fujifilm Canada,CAD,243.35
3,Fujifilm_FinePix_S1500,Fujifilm FinePix S1500FD 10 MP 12x Zoom Digital Camera (Black),Fujifilm Canada,CAD,243.35
4,Kodak_EasyShare_M575,Kodak EasyShare M575 14MP Digital Camera with 5x Wide Angle Optical Zoom and 3.0 Inch LCD (Midnight Black),Eastman Kodak Company,USD,70.00
5,Toshiba_PDR-M60,Toshiba PDR-M60 2.3MP Digital Camera w/ 2x Optical Zoom,Toshiba,USD,19.50
6,Sony_Alpha_NEX-3,Sony Alpha NEX-3 Interchangeable Lens Digital Camera w/18-55mm Lens (Red)- 14.2 Mpix,Sony,USD,521.99
7,Casio_Exilim_EX-Z29,Casio Exilim EX-Z29 10.1 MP Digital Camera with 3x Optical Zoom and 2.7-Inch LCD (Purple),CASIO,USD,59.98
8,Casio_Exilim_EX-FC100,Casio High-Speed Exilim EX-FC100 9.1 MP Digital Camera with 5x Optical Image Stabilized Zoom and 2.7-inch LCD (Black),CASIO,USD,179.99
9,Samsung_HZ15W,Samsung HZ15W 12MP Digital Camera with 10x Schneider Wide Angle Dual Image Stabilized Zoom and 3.0 inch LCD (Grey),Samsung,USD,182.99


In [78]:
f = open("Sortable.txt", "w")
for row in output.iterrows():
    row[1].to_json(f)
    f.write("\n")
f.close()